reference: https://github.com/edyoda/pyspark-tutorial

#### Intrduction to Spark

- At a high level, every Spark application consists of a driver program that runs the user’s main function and executes various parallel operations on a cluster.
- The main abstraction Spark provides is a resilient distributed dataset (RDD), which is a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel.
- RDDs are created by starting with a file in the Hadoop file system (or any other Hadoop-supported file system), or an existing Scala collection in the driver program, and transforming it.
- Users may also ask Spark to persist an RDD in memory, allowing it to be reused efficiently across parallel operations.
- Finally, RDDs automatically recover from node failures.

In [7]:
import pyspark
from pyspark import SparkContext

#### RDD

- A Resilient Distributed Dataset (RDD), the basic abstraction in Spark.
- Represents an immutable, partitioned collection of elements that can be operated on in parallel.

#### Two ways to create RDD

- parallelize -from collection
- textFile -from external file

In [8]:
sc = SparkContext("local", "count app")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=count app, master=local) created by __init__ at <ipython-input-2-52cd92508d6c>:1 

#### parallelize

In [ ]:
rdd = sc.parallelize([1,2,3,4,5],2)

In [9]:
rdd.collect()

[1, 2, 3, 4, 5]

#### External Datsets

In [10]:
baby_name = sc.textFile('data/Baby_Names_Beginning_2007.csv')

#### Basics of RDD

In [11]:
lines = sc.textFile('data/Baby_Names_Beginning_2007.csv')

lines.first()

'Year,First Name,County,Sex,Count'

In [12]:
lines.take(5)

['Year,First Name,County,Sex,Count',
 '2013,GAVIN,ST LAWRENCE,M,9',
 '2013,LEVI,ST LAWRENCE,M,9',
 '2013,LOGAN,NEW YORK,M,44',
 '2013,HUDSON,NEW YORK,M,49']

In [13]:
# length of first 5 elements
lines.map(lambda s: len(s)).take(5)

[32, 26, 25, 24, 25]

In [14]:
# return total number of characters
rdd = lines.map(lambda s: len(s))
rdd = rdd.map(lambda s: 2*s)
print (rdd.reduce(lambda a,b: a+b))

2424036


#### Key-Value Pairs RDD

In [15]:
rdd = sc.parallelize(["hello", "world", "good", "hello"])

In [16]:
rdd = rdd.map(lambda w: (w,1))
rdd.collect()

[('hello', 1), ('world', 1), ('good', 1), ('hello', 1)]

- value corresponding to same key undergoes lambda operation
- Note: Any function which has (key,value) pair can be worked on by {Any}ByKey

In [17]:
rdd.reduceByKey(lambda x, y: x+y).collect()

[('hello', 2), ('world', 1), ('good', 1)]

#### Transformation
- Eg - map, filter, flatMap ...
- Changes data from one format to another
- Lazy execution - Delays execution untill finds an 'Action' so that it can prepare optimized lineage ( spark internal code pipeline )

#### Actions
- Eg - count, collect, reduce ...
- Trigger execution of pipeline

#### Shuffle Operations

- Many operations in spark trigger shuffle .i.e movement of data across one one to another.
- Data movement is expensive & should be as less as possible

In [18]:
# Create 2 partition
rdd = sc.parallelize(["hello", "world", "good", "hello"],2)

In [19]:
# glom - retusn data in one partition in list
rdd. glom().collect()

[['hello', 'world'], ['good', 'hello']]

In [20]:
rdd = rdd.map(lambda w:(w,1))

In [21]:
rdd.glom().collect()

[[('hello', 1), ('world', 1)], [('good', 1), ('hello', 1)]]

In [22]:
# reduceByKey - generates a new RDD where all the values of same key are tupled 
rdd.reduceByKey(lambda a,b:(a,b)).collect()

[('world', 1), ('good', 1), ('hello', (1, 1))]

- The above operation brings all data with same key in one node
- This operation causes data shuffling

Note - We can reduce shuffle using groupByKey

#### RDD Operations

#### aggregate
- Aggregate the elements of each partition.
- Aggregate the result of each partition
- 'zero_value' isdefault init value

In [23]:
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x,y: (x[0] + y[0], x[1] + y[1]))

print (sc.parallelize([1, 2, 3, 4]).aggregate((0,0), seqOp, combOp))

print(sc.parallelize([]).aggregate((0,0), seqOp, combOp))

(10, 4)
(0, 0)


#### aggregateByKey
- seqOp works on each partition
- combOp works on result of each partitions
- ByKey causes operations on data with same key

In [24]:
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x,y: (x[0] + y[0], x[1] + y[1]))

In [25]:
sc.parallelize([('hello', 1), ('good', 2), ('hello', 3), ('food', 4)]).aggregateByKey((0,0), seqOp, combOp).collect()

[('hello', (4, 2)), ('good', (2, 1)), ('food', (4, 1))]

#### cache
- Prevent re-computation of RDD
- In-Memory caching wherever computation is happening

In [26]:
# Using cache() persisits
rdd = sc.parallelize(range(1000)) # first time this line be executed
rdd.cache()
rdd1 = rdd.map(lambda x: x+2)
rdd2 = rdd.map(lambda x: x+3)

In [27]:
rdd1.count()

1000

In [28]:
rdd2.count()

1000

In [29]:
# remove data from cache
rdd.unpersist()

PythonRDD[40] at RDD at PythonRDD.scala:53

#### Set Operation
- cartesian
- union
- intersection

In [30]:
rdd1 = sc.parallelize(range(1, 10))
rdd2 = sc.parallelize(range(11, 20))
rdd3 = sc.parallelize(range(5, 10))

In [31]:
rdd1.cartesian(rdd2).take(5)

[(1, 11), (1, 12), (1, 13), (1, 14), (1, 15)]

In [32]:
rdd1.intersection(rdd3).collect()

[6, 8, 5, 7, 9]

#### checkpoint
- Checkpoint current data of RDD
- Need to first set dir, where data will be persisted

In [33]:
sc.setCheckpointDir('ckpt')
rdd.checkpoint()

#### coalesce
- Return a new rdd with reduced partitions

In [34]:
sc.parallelize([1, 2, 3, 4, 5], 3).glom().collect()

[[1], [2, 3], [4, 5]]

In [35]:
sc.parallelize([1,2,3,4,5], 3).coalesce(1).glom().collect()

[[1, 2, 3, 4, 5]]

#### cogroup
- For each key k in self or other, return a resulting RDD that contains a tuple with the list of values for that key in self as well as other.

In [36]:
x = sc.parallelize([("a", 1), ("b", 4), ("a", 9)])
y = sc.parallelize([("a", 5)])

In [37]:
new_rdd = x.cogroup(y)

In [38]:
new_rdd.collect()

[('b',
  (<pyspark.resultiterable.ResultIterable at 0x1fd785a4648>,
   <pyspark.resultiterable.ResultIterable at 0x1fd785a4d48>)),
 ('a',
  (<pyspark.resultiterable.ResultIterable at 0x1fd785a43c8>,
   <pyspark.resultiterable.ResultIterable at 0x1fd785a4308>))]

In [39]:
[(a, map(list, b)) for a, b in new_rdd.collect()]

[('b', <map at 0x1fd7859cf88>), ('a', <map at 0x1fd7859c748>)]

#### collect
- Gets data from all executor nodes to driver
- Should be avoid if data is large

In [40]:
rdd = sc.parallelize([1,2,3,4],2)
rdd.collect()

[1, 2, 3, 4]

#### collectAsMap
- Works only on PairRDD
- Gets data to driver

In [41]:
m = sc.parallelize([(1, 2), (3, 4), (1, 6)]).collectAsMap()

In [42]:
m 

{1: 6, 3: 4}

#### countByValue
- Returns a dict with value & counter corresponding to it

In [43]:
sc.parallelize(['a', 'c', 'a', 'd', 'c']).countByValue()

defaultdict(int, {'a': 2, 'c': 2, 'd': 1})

#### combineByKey

In [44]:
# invoked per partition first time a key appears, d is the corresponding value
def mystr(d):
    print('In myStr')
    return d

# 2nd time & onwards for same key in same partition
def myconcat(a,b):
    print('In MyConcat')
    return a + b

# Workds across partitions
def mypartConcat(a,b):
    print('In myPartConcate')
    return a + b
    
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 2), ("a", 8), ("c", 4), ("a", 12), ("a", 18), ("c", 14)], 2)

#mystr 0 this converts the V into type C
rdd.combineByKey(mystr, myconcat, mypartConcat).collect()

[('b', 1), ('c', 18), ('a', 41)]

In [45]:
# Invoked per partition first time a key appears, d is the corresonging value 
def mystr(d):
    print('In MyStr')
    return (d,1)

# 2nd time & onwards for same key in same partition
def myconcat(a,b):
    print('In My Concat')
    return (a[0] + b, a[1] + 1)

# Work across partitions
def mypartConcatt(a,b):
    print ('In myPartConcat')
    return (a[0] + b[0], a[1] + b[1])

#mystr - this converts the V into of type C
data = rdd.combineByKey(mystr, myconcat, mypartConcat)
data.map(lambda x: (x[0], x[1][0]/(x[1][1]*1.0))).collect()

[('b', 1.0), ('c', 9.0), ('a', 3.6666666666666665)]

#### countByKey
- Count the number of elements for each key, and return the result to the master as a dictionary

In [46]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 3)])

In [47]:
rdd.countByKey()

defaultdict(int, {'a': 2, 'b': 1})

#### distinct
- Return a new RDD containing the distinct elements in this RDD.

In [48]:
sc.parallelize([1,1,2,3]).distinct().collect()

[1, 2, 3]

### filter
- selecting data conditionally

In [49]:
rdd = sc.parallelize(range(20))
rdd.filter(lambda x: x%2 != 0).map(lambda x: x*2).collect()

[2, 6, 10, 14, 18, 22, 26, 30, 34, 38]

#### first
- return first element of rdd

In [50]:
rdd.first()

0

#### flatMap
- Convert 1 data to N data

In [51]:
rdd = sc.parallelize([5, 6, 7, 8])
rdd.flatMap(lambda d: (d, d+1, d+2)).collect()

[5, 6, 7, 6, 7, 8, 7, 8, 9, 8, 9, 10]

#### foreach
- Applies a function to all elements of this RDD.

In [52]:
def f(e):
    print(e)

# This print happens in each executor & not on driver
# Chk on console if running on linux/aws
sc.parallelize([1, 2, 3, 4, 5]).foreach(f)

#### foreachPartition
- Applies function for each partition

In [53]:
def f(iterator):
    for x in iterator:
        print(x, )
        print( '\n Next Partition')

sc.parallelize([11, 12, 13, 14, 15], 2).foreachPartition(f)

#### getNumPartitions
- Returns number of partitions data is broken down into

In [54]:
rdd = sc.parallelize([11, 12, 13, 14, 15], 2)
rdd.getNumPartitions()

2

### StorageLevel
- MEMORY_ONLY Store RDD as deserialized Java objects in the JVM. If the RDD does not fit in memory, some partitions will not be cached and will be recomputed on the fly each time they're needed. This is the default level.
- MEMORY_AND_DISK Store RDD as deserialized Java objects in the JVM. If the RDD does not fit in memory, store the partitions that don't fit on disk, and read them from there when they're needed.
- DISK_ONLY Store the RDD partitions only on disk.
MEMORY_ONLY_2, MEMORY_AND_DISK_2 Same as the levels above, but replicate each partition on two cluster nodes.

#### getStorageLevel
- return rdd storage location


In [55]:
import pyspark

rdd1 = sc.parallelize([1,2])
rdd1. persist(storageLevel=pyspark.StorageLevel.MEMORY_AND_DISK)
print(rdd1.getStorageLevel())

Disk Memory Serialized 1x Replicated


#### randomSplit
- Split rdd elements into two parts

In [56]:
rdd = sc.parallelize(range(500), 1)
rdd1, rdd2 = rdd.randomSplit([2, 3], 17)
rdd1.count()

192

In [57]:
rdd2.count()

308

#### reduce
- Take two data & return one

In [58]:
from operator import add
sc.parallelize([1,2,3,4,5]).reduce(add)

15

In [59]:
sc.parallelize([1,2,3,4,5]).reduce(lambda a,b:a*b)

120

#### reduceByKey
- Merge the values for each key using an associative and commutative reduce function.
- This will also perform the merging locally on each mapper before sending results to a reducer, similarly to a “combiner” in MapReduce.

In [60]:
from operator import add
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.reduceByKey(add).collect())

[('a', 2), ('b', 1)]

#### repartition
- Return a new RDD that has exactly numPartitions partitions.
- Can increase or decrease the level of parallelism in this RDD.
- Internally, this uses a shuffle to redistribute data.
- If you are decreasing the number of partitions in this RDD, consider using coalesce, which can avoid performing a shuffle.

In [61]:
rdd = sc.parallelize([1,2,3,4,5,6,7], 4)
rdd.glom().collect()
rdd.repartition(2).glom().collect()

[[1, 4, 5, 6, 7], [2, 3]]

In [62]:
rdd.repartition(10).glom().collect() 

[[], [1], [4, 5, 6, 7], [2, 3], [], [], [], [], [], []]

#### saveAsTextFile
- save rdd into a text file

In [64]:
sc.parallelize(range(10)).saveAsTextFile('abcd.txt')

#### sortBy
- Sorts this RDD by the given keyfunc

In [65]:
tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]

In [66]:
sc.parallelize(tmp).sortBy(lambda x: x[0]).collect()

[('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]

#### sortByKey
- Sort based on keys

In [67]:
tmp2 = [('Mary', 1), ('had', 2), ('a', 3), ('little', 4), ('lamb', 5)]
tmp2.extend([('whose', 6), ('fleece', 7), ('was', 8), ('white', 9)])

In [68]:
sc.parallelize(tmp2).sortByKey(ascending=True, numPartitions=3, keyfunc=lambda k: k.lower()).collect()

[('a', 3),
 ('fleece', 7),
 ('had', 2),
 ('lamb', 5),
 ('little', 4),
 ('Mary', 1),
 ('was', 8),
 ('white', 9),
 ('whose', 6)]

#### subtract
- subtracts only when key value pair is same

In [69]:
x = sc.parallelize([("a", 1), ("b", 4), ("b", 5), ("a", 3)])
y = sc.parallelize([("a", 3), ("c", None)])
x.subtract(y).collect()

[('a', 1), ('b', 4), ('b', 5)]

#### subtractByKey
- Subtracts from the first rdd all the elements with the key present in second one

In [70]:
x = sc.parallelize([("a", 1), ("b", 4), ("b", 5), ("a", 2)])
y = sc.parallelize([("a", 3), ("c", None)])
x.subtractByKey(y).collect()

[('b', 4), ('b', 5)]

#### take
- return first n elements

In [71]:
sc.parallelize([2, 3, 4, 5, 6]).cache().take(2)

[2, 3]

#### takeordered
- Get the N elements from an RDD ordered in ascending order or as specified by the optional key function.

In [72]:
sc.parallelize([10, 1, 2, 9, 3, 4, 5, 6, 7], 2).takeOrdered(6, key=lambda x:-x)

[10, 9, 7, 6, 5, 4]

#### toDebugString
- A description of this RDD and its recursive dependencies for debugging.

In [73]:
rdd.toDebugString()

b'(4) ParallelCollectionRDD[120] at readRDDFromFile at PythonRDD.scala:262 []'

#### values
- Return value of each key-value pair

In [75]:
m = sc.parallelize([(1, 2), (3, 4)]).values()

In [76]:
m.collect()

[2, 4]

#### zip
- zips one rdd with another

In [77]:
x = sc.parallelize(range(0,5))
y = sc.parallelize(range(1000, 1005))

In [78]:
x.zip(y).collect()

[(0, 1000), (1, 1001), (2, 1002), (3, 1003), (4, 1004)]